In [1]:
import pandas as pd
import re
pd.set_option('precision', 0)
from sqlalchemy import create_engine

# Extracting Per Capita Personal Income (2018)

In [2]:
census_file = 'https://www.bea.gov/system/files/2019-11/lapi1119msa.xlsx'
# Local backup:
#census_file = 'data/lapi1119msa.xlsx'
census_df = pd.read_excel(census_file, index_col=0)

# Take a peek
census_df

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
"Table 2. Per Capita Personal Income by Metropolitan Area, 2016 - 2018",,,,,,,
NaN,Per capita personal income1,NaN,NaN,NaN,Percent change from preceding period,NaN,NaN
NaN,Dollars,NaN,NaN,Rank in United States,Percent change,NaN,Rank in United States
NaN,2016,2017,2018,2018,2017,2018,2018
United States,49870,51885,54446,--,4,5,--
Metropolitan portion,51742,53864,56527,--,4,5,--
...,...,...,...,...,...,...,...
"Yuba City, CA",40729,41270,42925,260,1,4,282
"Yuma, AZ",33141,35049,35682,374,6,2,380
1. Per capita personal income was computed using Census Bureau midyear population estimates. Estimates reflect county population estimates available as of March 2019.,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cleanup
This file needs to be cleaned up a little and the only thing we care about is the cities and the values for the year 2018

In [3]:
# Drop NaN rows
census_df = census_df.dropna(axis='rows')

# Drop the columns we don't need
census_df = census_df.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'])

# Drop the rows we don't need
census_df = census_df.drop(['United States','Metropolitan portion', 'Nonmetropolitan portion'])
census_df = census_df.iloc[1:]

# Rename the index to something more descriptive (and easy to call)
census_df.index.names = ['metro_area']

# Rename the column to something more descriptive (and easy to call)
census_df = census_df.rename(columns={'Unnamed: 3': 'per_capita_income'})

In [4]:
# confirm
census_df

,per_capita_income
metro_area,
"Abilene, TX",43140
"Akron, OH",49423
"Albany, GA",37500
"Albany-Lebanon, OR",42891
"Albany-Schenectady-Troy, NY",58104
...,...
"Yakima, WA",43379
"York-Hanover, PA",50113
"Youngstown-Warren-Boardman, OH-PA",42443


Note: Now we need to address the metro areas containing multiple cities and multiple entries to connect with the housing data we're going to combine on

In [5]:
# Build a copy to transform
census_transform_df = census_df.reset_index()

In [6]:
# Traverse the cities
for ind in census_transform_df.index: 
    #print(test_df['metro_area'][ind])
    input_str = census_transform_df['metro_area'][ind]
    
    # Remove the text from ''-' to ',' (keeps the primary city)
    input_str = re.sub(r'-.*,', '', input_str)
    
    # Remove instances where we have a '/' instead of '-'
    input_str = re.sub(r'/.*,', '', input_str)
    
    # Where we have '-' leftover, those are between states, so keep the first state only
    input_str = input_str.split('-')[0]
    
    # Write it back to the dataframe
    census_transform_df.at[ind, 'metro_area'] = input_str

In [7]:
# Now, we traverse the cities again to clean up some newly created issues
# We now have no commas between cities and states!

for ind in census_transform_df.index: 
    # Look for entries with no commas
    if census_transform_df['metro_area'][ind].find(',') == -1:
        #census_transform_df.at[ind, 'metro_area'] = census_transform_df['metro_area'][ind].replace(' ',', ')   
        
        # Find the last occurrance of a ' ' (this will be just before the state) and replace with a ','
        original_string = census_transform_df.at[ind, 'metro_area']
        last_space_index = original_string.rfind(' ')
        new_string = original_string[:last_space_index] + ", " + original_string[last_space_index+1:]
        
        # Write it back to the dataframe
        census_transform_df.at[ind, 'metro_area'] = new_string
       
    else:
        continue

In [8]:
# Confirm
census_transform_df.head(10)

,metro_area,per_capita_income
0,"Abilene, TX",43140
1,"Akron, OH",49423
2,"Albany, GA",37500
3,"Albany, OR",42891
4,"Albany, NY",58104
5,"Albuquerque, NM",42536
6,"Alexandria, LA",43995
7,"Allentown, PA",54120
8,"Altoona, PA",46743
9,"Amarillo, TX",46131


In [9]:
# Confirm it worked
#census_transform_df.to_csv('income.csv')

# Extracting Home Sales Data (2018)
Source: http://files.zillowstatic.com/research/public_v2/median_sale_price/Metro_median_sale_price_uc_SFRCondo_smoothed_month.csv

In [10]:
# Path to the Zillow Home Sales .CSV file
zillow = "data/zillow_sales_data.csv"

# Read our Zillow Home Sales data into pandas
zillow_df = pd.read_csv(zillow)
zillow_df.head(10)

,RegionID,SizeRank,RegionName,RegionType,StateName,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,...,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30
0,102001,0,United States,Country,NaN,195075,196575,198742,200333,201133,...,259167,256833,255833,257333,256500,257000,258667,262833,264167,266500
1,394913,1,"New York, NY",Msa,NY,NaN,NaN,NaN,NaN,NaN,...,428897,423230,418000,413667,415333,417333,420000,422500,424500,434500
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,503333,496333,491083,482750,469917,...,661333,653000,651667,651667,653667,657000,665333,668333,665667,667333
3,394463,3,"Chicago, IL",Msa,IL,248333,248333,251667,254333,256250,...,241000,235833,232167,231500,228167,226500,231500,244167,251667,255000
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,149667,153000,155667,158333,159329,...,274548,271648,269667,271333,269667,270333,271833,278500,281000,285500
5,394974,5,"Philadelphia, PA",Msa,PA,220996,225330,230000,237667,242667,...,262500,254483,248633,248633,245333,240667,240667,252000,261667,267103
6,394692,6,"Houston, TX",Msa,TX,152641,154467,157300,160667,161986,...,244937,241843,241500,244833,243333,243663,242738,248405,248741,253000
7,395209,7,"Washington, DC",Msa,DC,364582,368459,375126,381826,378953,...,415164,406831,405167,410000,406333,406333,411758,428424,434161,433736
8,394856,8,"Miami-Fort Lauderdale, FL",Msa,FL,267217,265667,265333,265667,265667,...,289333,289500,291167,293833,293000,293000,295000,302333,307333,314000
9,394347,9,"Atlanta, GA",Msa,GA,185617,188167,191167,193667,193000,...,255833,252500,252500,253650,252317,254150,257833,267500,268838,269505


In [11]:
# Remove any RegionType that is not a US Metro
zillow_df = zillow_df.loc[(zillow_df["RegionType"] == "Msa")]

zillow_df

,RegionID,SizeRank,RegionName,RegionType,StateName,2008-04-30,2008-05-31,2008-06-30,2008-07-31,2008-08-31,...,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30
1,394913,1,"New York, NY",Msa,NY,NaN,NaN,NaN,NaN,NaN,...,428897,423230,418000,413667,415333,417333,420000,422500,424500,434500
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,503333,496333,491083,482750,469917,...,661333,653000,651667,651667,653667,657000,665333,668333,665667,667333
3,394463,3,"Chicago, IL",Msa,IL,248333,248333,251667,254333,256250,...,241000,235833,232167,231500,228167,226500,231500,244167,251667,255000
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,149667,153000,155667,158333,159329,...,274548,271648,269667,271333,269667,270333,271833,278500,281000,285500
5,394974,5,"Philadelphia, PA",Msa,PA,220996,225330,230000,237667,242667,...,262500,254483,248633,248633,245333,240667,240667,252000,261667,267103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,394586,504,"Faribault, MN",Msa,MN,167308,180442,188675,192967,194633,...,254298,244965,236665,228333,233833,233500,235833,240333,244667,256661
117,394309,527,"Albemarle, NC",Msa,NC,114667,117667,125000,126500,122333,...,178250,177833,174167,178665,171332,174832,159833,171167,169342,189008
118,394742,595,"Kerrville, TX",Msa,TX,174279,170150,174500,170067,176233,...,231133,259217,257633,269917,247233,259317,264400,272360,280693,278326
119,395091,641,"Shelbyville, TN",Msa,TN,97150,102483,108067,108667,109250,...,180533,182367,179967,190133,189933,185300,178967,175833,190233,191533


In [12]:
# Obtain Column names
zillow_df.columns

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       '2008-04-30', '2008-05-31', '2008-06-30', '2008-07-31', '2008-08-31',
       ...
       '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31',
       '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30'],
      dtype='object', length=152)

In [13]:
# Reflect only 2018 housing data from the Zillow Home Sales dataframe
reduced_zillow_df = zillow_df[["RegionName", "2018-01-31", "2018-02-28", "2018-03-31", 
                               "2018-04-30", "2018-05-31", "2018-06-30", "2018-07-31", 
                               "2018-08-31", "2018-09-30", "2018-10-31", "2018-11-30", 
                               "2018-12-31"]]
reduced_zillow_df

,RegionName,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31
1,"New York, NY",398500,395500,394667,395000,401666,410000,420000,425333,425333,417000,411000,404333
2,"Los Angeles-Long Beach-Anaheim, CA",612666,615666,620667,632333,639333,646667,651667,651952,646952,643618,641667,637667
3,"Chicago, IL",218833,218500,222167,230000,237333,243667,245333,243300,236633,229754,226121,222788
4,"Dallas-Fort Worth, TX",251921,253588,255482,261661,268327,273333,276667,274000,267333,261333,258663,259997
5,"Philadelphia, PA",228300,225667,224000,229000,235000,241667,246667,251000,247833,242500,237467,235633
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,"Faribault, MN",208000,208667,206967,209300,208800,214667,218967,226235,228402,222586,217117,217451
117,"Albemarle, NC",141000,139167,143250,152250,149083,149000,144000,151083,152083,156750,153667,156667
118,"Kerrville, TX",235311,232145,232145,234167,245667,258667,260750,252083,235250,236667,245750,258750
119,"Shelbyville, TN",161400,165353,161120,161453,162267,163267,164100,164833,177167,177500,172833,159467


In [14]:
# Rename Column Headers and Include 2018 Sales Average 
final_zillow_df = pd.DataFrame(columns=["metro_area", "median_home_price_1_18", 
                                     "median_home_price_2_18", "median_home_price_3_18", 
                                     "median_home_price_4_18", "median_home_price_5_18", 
                                     "median_home_price_6_18", "median_home_price_7_18", 
                                     "median_home_price_8_18", "median_home_price_9_18", 
                                     "median_home_price_10_18", "median_home_price_11_18", 
                                     "median_home_price_12_18",])

final_zillow_df["metro_area"] = reduced_zillow_df["RegionName"].values
final_zillow_df["median_home_price_1_18"] = reduced_zillow_df["2018-01-31"].values
final_zillow_df["median_home_price_2_18"] = reduced_zillow_df["2018-02-28"].values
final_zillow_df["median_home_price_3_18"] = reduced_zillow_df["2018-03-31"].values
final_zillow_df["median_home_price_4_18"] = reduced_zillow_df["2018-04-30"].values
final_zillow_df["median_home_price_5_18"] = reduced_zillow_df["2018-05-31"].values
final_zillow_df["median_home_price_6_18"] = reduced_zillow_df["2018-06-30"].values
final_zillow_df["median_home_price_7_18"] = reduced_zillow_df["2018-07-31"].values
final_zillow_df["median_home_price_8_18"] = reduced_zillow_df["2018-08-31"].values
final_zillow_df["median_home_price_9_18"] = reduced_zillow_df["2018-09-30"].values
final_zillow_df["median_home_price_10_18"] = reduced_zillow_df["2018-10-31"].values
final_zillow_df["median_home_price_11_18"] = reduced_zillow_df["2018-11-30"].values
final_zillow_df["median_home_price_12_18"] = reduced_zillow_df["2018-12-31"].values

final_zillow_df

,metro_area,median_home_price_1_18,median_home_price_2_18,median_home_price_3_18,median_home_price_4_18,median_home_price_5_18,median_home_price_6_18,median_home_price_7_18,median_home_price_8_18,median_home_price_9_18,median_home_price_10_18,median_home_price_11_18,median_home_price_12_18
0,"New York, NY",398500,395500,394667,395000,401666,410000,420000,425333,425333,417000,411000,404333
1,"Los Angeles-Long Beach-Anaheim, CA",612666,615666,620667,632333,639333,646667,651667,651952,646952,643618,641667,637667
2,"Chicago, IL",218833,218500,222167,230000,237333,243667,245333,243300,236633,229754,226121,222788
3,"Dallas-Fort Worth, TX",251921,253588,255482,261661,268327,273333,276667,274000,267333,261333,258663,259997
4,"Philadelphia, PA",228300,225667,224000,229000,235000,241667,246667,251000,247833,242500,237467,235633
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,"Faribault, MN",208000,208667,206967,209300,208800,214667,218967,226235,228402,222586,217117,217451
116,"Albemarle, NC",141000,139167,143250,152250,149083,149000,144000,151083,152083,156750,153667,156667
117,"Kerrville, TX",235311,232145,232145,234167,245667,258667,260750,252083,235250,236667,245750,258750
118,"Shelbyville, TN",161400,165353,161120,161453,162267,163267,164100,164833,177167,177500,172833,159467


In [15]:
# Combine Multi City Metro's to match with the income data
# Build a copy to transform
zillow_transform_df = final_zillow_df.copy()

In [16]:
# Traverse the cities
for ind in zillow_transform_df.index: 
    
    #print(test_df['metro_area'][ind])
    input_str = zillow_transform_df['metro_area'][ind]
    
    # Remove the text from ''-' to ',' (keeps the primary city)
    input_str = re.sub(r'-.*,', '', input_str)
    
    # Write it back to the dataframe
    zillow_transform_df.at[ind, 'metro_area'] = input_str

In [17]:
# Now, we traverse the cities again to clean up some newly created issues
# We now have no commas between cities and states
for ind in zillow_transform_df.index: 
    # Look for entries with no commas
    if zillow_transform_df['metro_area'][ind].find(',') == -1:
        #census_transform_df.at[ind, 'metro_area'] = census_transform_df['metro_area'][ind].replace(' ',', ')   
        
        # Find the last occurrance of a ' ' (this will be just before the state) and replace with a ','
        original_string = zillow_transform_df.at[ind, 'metro_area']
        last_space_index = original_string.rfind(' ')
        new_string = original_string[:last_space_index] + ", " + original_string[last_space_index+1:]
        
        # Write it back to the dataframe
        zillow_transform_df.at[ind, 'metro_area'] = new_string
       
    else:
        continue
zillow_transform_df.head(10)

,metro_area,median_home_price_1_18,median_home_price_2_18,median_home_price_3_18,median_home_price_4_18,median_home_price_5_18,median_home_price_6_18,median_home_price_7_18,median_home_price_8_18,median_home_price_9_18,median_home_price_10_18,median_home_price_11_18,median_home_price_12_18
0,"New York, NY",398500,395500,394667,395000,401666,410000,420000,425333,425333,417000,411000,404333
1,"Los Angeles, CA",612666,615666,620667,632333,639333,646667,651667,651952,646952,643618,641667,637667
2,"Chicago, IL",218833,218500,222167,230000,237333,243667,245333,243300,236633,229754,226121,222788
3,"Dallas, TX",251921,253588,255482,261661,268327,273333,276667,274000,267333,261333,258663,259997
4,"Philadelphia, PA",228300,225667,224000,229000,235000,241667,246667,251000,247833,242500,237467,235633
5,"Houston, TX",224089,223756,224323,231330,237497,241497,242500,240907,236907,232636,230729,234063
6,"Washington, DC",387982,388000,388000,398000,409333,415152,417318,414652,407500,399098,394431,399098
7,"Miami, FL",270000,269000,270667,274000,278333,283333,285582,285582,280582,280000,281667,283333
8,"Atlanta, GA",228300,229967,231100,237216,243648,247797,248681,245549,244133,243433,243670,242837
9,"Boston, MA",422300,420646,424646,436480,448133,462467,469967,473333,458333,444667,438000,435667


In [18]:
#Add average column
sales_average_df = zillow_transform_df.copy()
sales_average_df['sales_average_2018'] = zillow_transform_df.sum(axis=1) / 12
pd.set_option('precision', 0)
sales_average_df.head()

,metro_area,median_home_price_1_18,median_home_price_2_18,median_home_price_3_18,median_home_price_4_18,median_home_price_5_18,median_home_price_6_18,median_home_price_7_18,median_home_price_8_18,median_home_price_9_18,median_home_price_10_18,median_home_price_11_18,median_home_price_12_18,sales_average_2018
0,"New York, NY",398500,395500,394667,395000,401666,410000,420000,425333,425333,417000,411000,404333,408194
1,"Los Angeles, CA",612666,615666,620667,632333,639333,646667,651667,651952,646952,643618,641667,637667,636738
2,"Chicago, IL",218833,218500,222167,230000,237333,243667,245333,243300,236633,229754,226121,222788,231202
3,"Dallas, TX",251921,253588,255482,261661,268327,273333,276667,274000,267333,261333,258663,259997,263525
4,"Philadelphia, PA",228300,225667,224000,229000,235000,241667,246667,251000,247833,242500,237467,235633,237061


In [19]:
# Confirm it worked
#sales_average_df.to_csv('sales.csv')

## Extracting Rental Data (2018)
Source: http://files.zillowstatic.com/research/public_v2/zori/Metro_ZORI_AllHomesPlusMultifamily_Smoothed.csv

In [20]:
#read in csv data
rent_data = pd.read_csv('data/Zillow_Rental.csv')
rent_data.head()

,RegionID,RegionName,SizeRank,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,...,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07
0,102001,United States,0,1389,1396,1399,1415,1426,1437,1446,...,1728,1726,1730,1738,1748,1755,1750,1748,1748,1749
1,394913,"New York, NY",1,2418,2432,2442,2472,2495,2511,2521,...,2837,2819,2811,2819,2845,2855,2831,2816,2796,2758
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,1889,1925,1922,1944,1965,1992,2017,...,2594,2599,2606,2623,2630,2630,2608,2588,2594,2600
3,394463,"Chicago, IL",3,1550,1570,1566,1584,1598,1610,1618,...,1808,1800,1797,1809,1817,1826,1825,1825,1824,1821
4,394514,"Dallas-Fort Worth, TX",4,1270,1263,1271,1284,1298,1311,1323,...,1614,1608,1606,1612,1623,1627,1618,1614,1620,1628


In [21]:
#drop all yearly data except 2018
rent_df = rent_data.drop(columns=['2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
                                   '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
                                   '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
                                   '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
                                   '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
                                   '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12',
                                   '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
                                   '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
                                   '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
                                   '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                                   '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
                                   '2020-07'])

rent_df.head()

,RegionID,RegionName,SizeRank,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,102001,United States,0,1619,1629,1636,1645,1655,1663,1670,1673,1674,1674,1675,1672
1,394913,"New York, NY",1,2688,2707,2717,2732,2747,2759,2766,2768,2765,2759,2749,2737
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2433,2449,2452,2463,2476,2487,2495,2513,2522,2529,2533,2529
3,394463,"Chicago, IL",3,1726,1737,1746,1755,1770,1780,1787,1784,1773,1760,1751,1751
4,394514,"Dallas-Fort Worth, TX",4,1518,1527,1529,1532,1541,1553,1561,1559,1561,1568,1570,1562


In [22]:
#drop United States row
rent_df = rent_df.drop([0])
rent_df.head()

,RegionID,RegionName,SizeRank,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
1,394913,"New York, NY",1,2688,2707,2717,2732,2747,2759,2766,2768,2765,2759,2749,2737
2,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2433,2449,2452,2463,2476,2487,2495,2513,2522,2529,2533,2529
3,394463,"Chicago, IL",3,1726,1737,1746,1755,1770,1780,1787,1784,1773,1760,1751,1751
4,394514,"Dallas-Fort Worth, TX",4,1518,1527,1529,1532,1541,1553,1561,1559,1561,1568,1570,1562
5,394974,"Philadelphia, PA",5,1503,1515,1523,1533,1544,1547,1548,1551,1554,1556,1557,1557


In [23]:
#reset index so first result is [0]
rent_df = rent_df.reset_index(drop=True)
rent_df.head()

,RegionID,RegionName,SizeRank,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,394913,"New York, NY",1,2688,2707,2717,2732,2747,2759,2766,2768,2765,2759,2749,2737
1,753899,"Los Angeles-Long Beach-Anaheim, CA",2,2433,2449,2452,2463,2476,2487,2495,2513,2522,2529,2533,2529
2,394463,"Chicago, IL",3,1726,1737,1746,1755,1770,1780,1787,1784,1773,1760,1751,1751
3,394514,"Dallas-Fort Worth, TX",4,1518,1527,1529,1532,1541,1553,1561,1559,1561,1568,1570,1562
4,394974,"Philadelphia, PA",5,1503,1515,1523,1533,1544,1547,1548,1551,1554,1556,1557,1557


In [24]:
#further column consolidation
rent_df = rent_df.drop(['RegionID', 'SizeRank'], axis=1)
rent_df = rent_df.rename({"RegionName": "metro_area", "2018-01": "median_rent_price_1_18", "2018-02": "median_rent_price_2_18",
                "2018-03" : "median_rent_price_3_18", "2018-04" : "median_rent_price_4_18", "2018-05" : "median_rent_price_5_18",
                "2018-06" : "median_rent_price_6_18", "2018-07" : "median_rent_price_7_18", "2018-08" : "median_rent_price_8_18",
                "2018-09" : "median_rent_price_9_18", "2018-10" : "median_rent_price_10_18", "2018-11" : "median_rent_price_11_18",
                "2018-12" : "median_rent_price_12_18"}, axis=1)
rent_df.head()

,metro_area,median_rent_price_1_18,median_rent_price_2_18,median_rent_price_3_18,median_rent_price_4_18,median_rent_price_5_18,median_rent_price_6_18,median_rent_price_7_18,median_rent_price_8_18,median_rent_price_9_18,median_rent_price_10_18,median_rent_price_11_18,median_rent_price_12_18
0,"New York, NY",2688,2707,2717,2732,2747,2759,2766,2768,2765,2759,2749,2737
1,"Los Angeles-Long Beach-Anaheim, CA",2433,2449,2452,2463,2476,2487,2495,2513,2522,2529,2533,2529
2,"Chicago, IL",1726,1737,1746,1755,1770,1780,1787,1784,1773,1760,1751,1751
3,"Dallas-Fort Worth, TX",1518,1527,1529,1532,1541,1553,1561,1559,1561,1568,1570,1562
4,"Philadelphia, PA",1503,1515,1523,1533,1544,1547,1548,1551,1554,1556,1557,1557


In [25]:
#Handling of Multi-city named metro_areas
rent_transform_df = rent_df.copy()

for ind in rent_transform_df.index:
    input_str = rent_transform_df['metro_area'][ind]
    input_str = re.sub(r'-.*,', '', input_str)
    rent_transform_df.at[ind, 'metro_area'] =input_str
    
#confirming missing comma issue
rent_transform_df.head(10)

,metro_area,median_rent_price_1_18,median_rent_price_2_18,median_rent_price_3_18,median_rent_price_4_18,median_rent_price_5_18,median_rent_price_6_18,median_rent_price_7_18,median_rent_price_8_18,median_rent_price_9_18,median_rent_price_10_18,median_rent_price_11_18,median_rent_price_12_18
0,"New York, NY",2688,2707,2717,2732,2747,2759,2766,2768,2765,2759,2749,2737
1,Los Angeles CA,2433,2449,2452,2463,2476,2487,2495,2513,2522,2529,2533,2529
2,"Chicago, IL",1726,1737,1746,1755,1770,1780,1787,1784,1773,1760,1751,1751
3,Dallas TX,1518,1527,1529,1532,1541,1553,1561,1559,1561,1568,1570,1562
4,"Philadelphia, PA",1503,1515,1523,1533,1544,1547,1548,1551,1554,1556,1557,1557
5,"Houston, TX",1500,1506,1510,1525,1522,1526,1534,1532,1528,1523,1519,1512
6,"Washington, DC",2078,2086,2087,2106,2122,2135,2145,2145,2141,2137,2133,2127
7,Miami FL,1888,1895,1892,1898,1898,1906,1913,1920,1927,1934,1941,1939
8,"Atlanta, GA",1431,1442,1449,1457,1471,1480,1491,1500,1509,1514,1516,1515
9,"Boston, MA",2446,2466,2483,2498,2517,2526,2533,2521,2506,2503,2491,2490


In [26]:
#replacing missing comma

for ind in rent_transform_df.index:
    if rent_transform_df['metro_area'][ind].find(',') == -1:
        original_string = rent_transform_df.at[ind, 'metro_area']
        last_space_index = original_string.rfind(' ')
        new_string = original_string[:last_space_index] + ", " + original_string[last_space_index+1:]
        rent_transform_df.at[ind, 'metro_area'] = new_string
    else:
        continue

# confirm
rent_transform_df.head(10)

,metro_area,median_rent_price_1_18,median_rent_price_2_18,median_rent_price_3_18,median_rent_price_4_18,median_rent_price_5_18,median_rent_price_6_18,median_rent_price_7_18,median_rent_price_8_18,median_rent_price_9_18,median_rent_price_10_18,median_rent_price_11_18,median_rent_price_12_18
0,"New York, NY",2688,2707,2717,2732,2747,2759,2766,2768,2765,2759,2749,2737
1,"Los Angeles, CA",2433,2449,2452,2463,2476,2487,2495,2513,2522,2529,2533,2529
2,"Chicago, IL",1726,1737,1746,1755,1770,1780,1787,1784,1773,1760,1751,1751
3,"Dallas, TX",1518,1527,1529,1532,1541,1553,1561,1559,1561,1568,1570,1562
4,"Philadelphia, PA",1503,1515,1523,1533,1544,1547,1548,1551,1554,1556,1557,1557
5,"Houston, TX",1500,1506,1510,1525,1522,1526,1534,1532,1528,1523,1519,1512
6,"Washington, DC",2078,2086,2087,2106,2122,2135,2145,2145,2141,2137,2133,2127
7,"Miami, FL",1888,1895,1892,1898,1898,1906,1913,1920,1927,1934,1941,1939
8,"Atlanta, GA",1431,1442,1449,1457,1471,1480,1491,1500,1509,1514,1516,1515
9,"Boston, MA",2446,2466,2483,2498,2517,2526,2533,2521,2506,2503,2491,2490


In [27]:
#add column to average each row's values
rent_average_df = rent_transform_df.copy()
rent_average_df['rent_average_2018'] = rent_transform_df.sum(axis=1) / 12
pd.set_option('precision', 0)

# confirm
rent_average_df.head()

,metro_area,median_rent_price_1_18,median_rent_price_2_18,median_rent_price_3_18,median_rent_price_4_18,median_rent_price_5_18,median_rent_price_6_18,median_rent_price_7_18,median_rent_price_8_18,median_rent_price_9_18,median_rent_price_10_18,median_rent_price_11_18,median_rent_price_12_18,rent_average_2018
0,"New York, NY",2688,2707,2717,2732,2747,2759,2766,2768,2765,2759,2749,2737,2741
1,"Los Angeles, CA",2433,2449,2452,2463,2476,2487,2495,2513,2522,2529,2533,2529,2490
2,"Chicago, IL",1726,1737,1746,1755,1770,1780,1787,1784,1773,1760,1751,1751,1760
3,"Dallas, TX",1518,1527,1529,1532,1541,1553,1561,1559,1561,1568,1570,1562,1548
4,"Philadelphia, PA",1503,1515,1523,1533,1544,1547,1548,1551,1554,1556,1557,1557,1541


In [28]:
# Confirm it worked
#rent_average_df.to_csv('rent.csv')

# Connect to database

In [29]:
# Connect to Local Database
# Make sure you fill out the user / password for MySQL
rds_connection_string = "USER:PASSWORD@localhost:5432/2018_Housing_Income"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [30]:
# Check for Tables
engine.table_names()

['income', 'sales', 'rent']

In [31]:
# Use pandas to load converted income DataFrame into database
census_transform_df.to_sql(name='income', con=engine, if_exists='append', index=False) 

In [32]:
# Use pandas to load converted sales DataFrame into database
sales_average_df.to_sql(name='sales', con=engine, if_exists='append', index=False)

In [33]:
# Use pandas to load converted rent DataFrame into database
rent_average_df.to_sql(name='rent', con=engine, if_exists='append', index=False)

In [34]:
# Confirm Data Has Been Added By Querying income Dataframe
pd.read_sql_query('select * from income', con=engine).head() 

,metro_area,per_capita_income
0,"Abilene, TX",43140
1,"Akron, OH",49423
2,"Albany, GA",37500
3,"Albany, OR",42891
4,"Albany, NY",58104


In [35]:
# Confirm Data Has Been Added By Querying sales Dataframe
pd.read_sql_query('select * from sales', con=engine).head() 

,metro_area,median_home_price_1_18,median_home_price_2_18,median_home_price_3_18,median_home_price_4_18,median_home_price_5_18,median_home_price_6_18,median_home_price_7_18,median_home_price_8_18,median_home_price_9_18,median_home_price_10_18,median_home_price_11_18,median_home_price_12_18,sales_average_2018
0,"New York, NY",398500,395500,394667,395000,401666,410000,420000,425333,425333,417000,411000,404333,408194
1,"Los Angeles, CA",612666,615666,620667,632333,639333,646667,651667,651952,646952,643618,641667,637667,636738
2,"Chicago, IL",218833,218500,222167,230000,237333,243667,245333,243300,236633,229754,226121,222788,231202
3,"Dallas, TX",251921,253588,255482,261661,268327,273333,276667,274000,267333,261333,258663,259997,263525
4,"Philadelphia, PA",228300,225667,224000,229000,235000,241667,246667,251000,247833,242500,237467,235633,237061


In [36]:
# Confirm Data Has Been Added By Querying rent Dataframe
pd.read_sql_query('select * from rent', con=engine).head() 

,metro_area,median_rent_price_1_18,median_rent_price_2_18,median_rent_price_3_18,median_rent_price_4_18,median_rent_price_5_18,median_rent_price_6_18,median_rent_price_7_18,median_rent_price_8_18,median_rent_price_9_18,median_rent_price_10_18,median_rent_price_11_18,median_rent_price_12_18,rent_average_2018
0,"New York, NY",2688,2707,2717,2732,2747,2759,2766,2768,2765,2759,2749,2737,2741
1,"Los Angeles, CA",2433,2449,2452,2463,2476,2487,2495,2513,2522,2529,2533,2529,2490
2,"Chicago, IL",1726,1737,1746,1755,1770,1780,1787,1784,1773,1760,1751,1751,1760
3,"Dallas, TX",1518,1527,1529,1532,1541,1553,1561,1559,1561,1568,1570,1562,1548
4,"Philadelphia, PA",1503,1515,1523,1533,1544,1547,1548,1551,1554,1556,1557,1557,1541
